In [20]:
import csv 
import os

In [21]:
dataset = "../dataset/"

def get_lines(file_name, line_count, delimiter=","):
    lines = []
    with open(file_name, "r", encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=delimiter)
        for i, line in enumerate(reader):
            if i == line_count:
                break
            lines.append(line)
    return lines

In [23]:
pairs = get_lines(os.path.join(dataset, "pairs.csv"), 10)
for line in pairs:
    print(line)

['id_1', 'name_1', 'latitude_1', 'longitude_1', 'address_1', 'city_1', 'state_1', 'zip_1', 'country_1', 'url_1', 'phone_1', 'categories_1', 'id_2', 'name_2', 'latitude_2', 'longitude_2', 'address_2', 'city_2', 'state_2', 'zip_2', 'country_2', 'url_2', 'phone_2', 'categories_2', 'match']
['E_000001272c6c5d', 'Café Stad Oudenaarde', '50.85997533509319', '3.6341963487748346', 'Abdijstraat', 'Nederename', 'Oost-Vlaanderen', '9700', 'BE', '', '', 'Bars', 'E_da7fa3963561f8', 'Café Oudenaarde', '50.869358', '3.635206', '', '', '', '', 'BE', '', '', 'Bars', 'True']
['E_000008a8ba4f48', 'Turkcell', '37.84451021530735', '27.84420157619721', 'Adnan Menderes Bulvarı', '', '', '', 'TR', '', '', 'Mobile Phone Shops', 'E_3f12df0b045062', 'Turkcell', '37.83802', '27.84556', 'batı aydın', 'aydın', '', '67500', 'TR', '', '', 'Electronics Stores', 'False']
['E_000023d8f4be44', 'Island Spa', '14.518969876756294', '121.01870186448409', '5th Flr, Newport Mall, Resorts World Manila', 'Pasay City', 'Metro Man